# Forest Inventory Postprocessing
The Austrian Forest Inventory (BWF) is done in periods. Available were four periods that were 3-7 years long. 
The last year of each period is assumed as the fixed/given value. The periods get split into single years and missing years are filled up incrementally until the level of the next period is reached. 

## Presettings

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import os
import re

In [27]:
# All directories
root       = "C:/Users/freiste/OneDrive - Ilmatieteen laitos/Documents/IIASA YSSP 2023"
this_dir   =  os.getcwd()

input_dir1 = f"{root}/03 - Research and ForgetMeNots"
input_dir2 = f"{root}/02 - Data/AUT"
output_dir = input_dir2

## Global Methods

In [28]:
# Default value of display.max_rows is 10, so max 10 rows will be printed.
# Set it None to display all rows in the dataframe

def show_entire_df(switch = True):
    
    if switch == True:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
    else:
        pd.set_option('display.max_rows', 10)
        pd.set_option('display.max_columns', 10)


In [29]:
def check_empty_cells(column):
    
    empty_cells = []
    
    for i, el in enumerate(column):
        
        if type(el) == str:
            el = el.strip(" - NaN")
            if el == '':
                empty_cells.append(i)
        
    return empty_cells

In [30]:
def correct_empty_cells(column):
    
    suspects = check_empty_cells(column)
    
    for i in suspects:

        column.at[i] = np.nan

    return column

In [31]:
# Fill in missing years
def resample_years(df):
    mux = pd.MultiIndex.from_product([ df.ForestryDistrict.unique() , range(df.Year.min(), df.Year.max() + 1)], 
                                     names=['ForestryDistrict', 'Year'])

    return df.set_index(['ForestryDistrict', 'Year']).reindex(mux).reset_index()

In [32]:
def interval_filling(x, var):
    
    # List of indices where rows have data
    idxs = x[x[var] == x[var]].index

    pdseries = {}
    
    # if there is only one data point
    if len(idxs) == 1:
        return (x.index[0], x.index[-1], x[var].ffill().bfill())
    
    
    # if there is no data
    elif len(idxs) == 0:
        return (x.index[0], x.index[-1], np.nan)
    
    
    # if there are intervals between data
    else:
        for i, idx1 in enumerate(idxs[:-1]): 
            try:
                idx2 = idxs[i+1]

                y1   = x.loc[idx1].Year
                y2   = x.loc[idx2].Year

                val1 = x.loc[idx1][var]
                val2 = x.loc[idx2][var]


                if val1 == val2:
                    data = np.array([val1] * ((idx2-idx1)+1))
                    
                    pdseries[i] = pd.Series(data = data, index = range(idx1,idx2+1))

                else:
                    step = (val2-val1) / (y2-y1)

                    data = np.arange(start = val1, stop  = val2 , step  = step)

                    pdseries[i] = pd.Series(data = data, index = range(idx1, idx1 + len(data)))

                    if idx2 == idxs[-1]:
                        break
                    else:
                        pass
            
            except:
                print('error1')
                
                
                
        try:
            pds =  pd.concat([pdseries[i] for i in pdseries])
            pds =  pds[~pds.index.duplicated(keep="first")]
            return (pds.index[0], pds.index[-1], pds)
        
        except:
            print('error2')
            pass
        

## Import

In [33]:
# Forest inventory (BWF)     (WI)

FI = pd.read_excel(f'{input_dir2}/Data_BWF_ForestInventory.xlsx', 
                   sheet_name='ForestStructureAndDamages',
                   usecols=list(range(32)))

## Postprocess

In [34]:
FI.drop(index=[0,1,334], inplace=True)  # drop source and unit lines


# Fix the Period -> Year issue
FI.rename(columns={'Year':'Period'}, inplace=True)

FI_periods = ['1992-1996',  '2000-2002', '2007-2009', '2015-2021']
FI_periods_2 = [1996, 2002, 2009, 2021]

FI['Year'] = FI.Period
FI['Year'].replace(FI_periods, FI_periods_2, inplace=True)

# reorder the dataframe
order = ['ForestryDistrict', 'FederalState', 'Period', 'PeriodLength', 'Year'] + list(FI.columns)[4:-1]
FI = FI[order]
FI = FI.sort_values(['FederalState', 'ForestryDistrict', 'Year']).reset_index(drop=True)

# Find and correct problematic cells and convert to float
for var in FI.columns[5:]:
    
    print(var, check_empty_cells(FI[var]))
    correct_empty_cells(FI[var])
    
    FI[var] = FI[var].astype(float)


FI

Area []
DistrictShareAustrForest []
DistrictShareStateForest []
TotalForestShare []
TotalForestArea [136, 137]
ErtragswaldShare []
ErtragswaldArea []
ErtragswaldStock []
ErtragswaldTotalStock []
ConiferousShare []
ConiferousShare2 []
ConiferousArea []
ConiferousStock []
SpruceShareTotalArea []
SpruceShareTotalForest []
SpruceEWShare []
SpruceConifShare []
SpruceArea []
SpruceArea-SD [90, 98, 158, 322]
SpruceArea2 []
SpruceStockShare []
SpruceStock []
DWStShare []
DWStStock []
DWSt-SD [131, 139, 163, 167, 191, 323, 327]
DeadSpruceStShare []
DeadSpruceArea []
DeadSpruceDensity []


,ForestryDistrict,FederalState,Period,PeriodLength,Year,Area,DistrictShareAustrForest,DistrictShareStateForest,TotalForestShare,TotalForestArea,...,SpruceArea-SD,SpruceArea2,SpruceStockShare,SpruceStock,DWStShare,DWStStock,DWSt-SD,DeadSpruceStShare,DeadSpruceArea,DeadSpruceDensity
0,Eisenstadt+Rust,Burgenland,1992-1996,5,1996,516.03,0.004017,0.122137,0.310059,16000.000000,...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Eisenstadt+Rust,Burgenland,2000-2002,3,2002,516.03,0.004115,0.122180,0.314904,16250.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eisenstadt+Rust,Burgenland,2007-2009,3,2009,516.03,0.004143,0.123134,0.319749,16500.000000,...,500.0,1904.761905,NaN,400000.0,NaN,6.8,NaN,NaN,NaN,NaN
3,Eisenstadt+Rust,Burgenland,2015-2021,7,2021,516.03,0.003498,0.106478,0.270000,13932.810000,...,NaN,5139.097744,0.213627,341750.0,0.028,5.9,1.3,0.002800,3.706127,0.016520
4,Güssing,Burgenland,1992-1996,5,1996,485.34,0.005607,0.170483,0.460159,22333.333333,...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,Feldkirch,Vorarlberg,2015-2021,7,2021,278.31,0.003221,0.131858,0.461000,12830.091000,...,NaN,0.000000,NaN,NaN,0.018,6.7,NaN,0.011862,72.010780,0.079475
328,Vorarlberg,Vorarlberg_total,1992-1996,5,1996,2601.67,0.023600,1.000000,0.361306,94000.000000,...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329,Vorarlberg,Vorarlberg_total,2000-2002,3,2002,2601.67,0.024563,1.000000,0.372837,97000.000000,...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330,Vorarlberg,Vorarlberg_total,2007-2009,3,2009,2601.67,0.024354,1.000000,0.372837,97000.000000,...,4000.0,27855.345622,NaN,15499000.0,NaN,11.1,NaN,NaN,NaN,NaN


In [35]:
# Insert missing years

FI = resample_years(FI)    

for var in 'FederalState 	Period 	PeriodLength 	Area'.split(' 	'):
    FI[var] = FI[var].ffill()

FI

,ForestryDistrict,Year,FederalState,Period,PeriodLength,Area,DistrictShareAustrForest,DistrictShareStateForest,TotalForestShare,TotalForestArea,...,SpruceArea-SD,SpruceArea2,SpruceStockShare,SpruceStock,DWStShare,DWStStock,DWSt-SD,DeadSpruceStShare,DeadSpruceArea,DeadSpruceDensity
0,Eisenstadt+Rust,1996,Burgenland,1992-1996,5,516.03,0.004017,0.122137,0.310059,16000.000,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Eisenstadt+Rust,1997,Burgenland,1992-1996,5,516.03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eisenstadt+Rust,1998,Burgenland,1992-1996,5,516.03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Eisenstadt+Rust,1999,Burgenland,1992-1996,5,516.03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Eisenstadt+Rust,2000,Burgenland,1992-1996,5,516.03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2153,Vorarlberg,2017,Vorarlberg_total,2007-2009,3,2601.67,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2154,Vorarlberg,2018,Vorarlberg_total,2007-2009,3,2601.67,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2155,Vorarlberg,2019,Vorarlberg_total,2007-2009,3,2601.67,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2156,Vorarlberg,2020,Vorarlberg_total,2007-2009,3,2601.67,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
for var in FI.columns[6:]:
    print(var)
    
    id1, id2, fill = interval_filling(FI, var)
    
    FI.loc[slice(id1, id2), var] = fill
    

DistrictShareAustrForest
DistrictShareStateForest
TotalForestShare
TotalForestArea
ErtragswaldShare
ErtragswaldArea
ErtragswaldStock
ErtragswaldTotalStock
ConiferousShare


C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)
C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)
C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)
C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)


ConiferousShare2
ConiferousArea
ConiferousStock
SpruceShareTotalArea


C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)
C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)
C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)


SpruceShareTotalForest
SpruceEWShare
SpruceConifShare


...

C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)
C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)


SpruceStock
DWStShare
DWStStock


C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)
C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)


DWSt-SD
DeadSpruceStShare
DeadSpruceArea
DeadSpruceDensity


C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)
C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)
C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)
C:\Users\freiste\AppData\Local\Temp\ipykernel_20796\2167575744.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  step = (val2-val1) / (y2-y1)


In [37]:
# Fill up the rest
fd = FI['ForestryDistrict']
FI = FI.groupby('ForestryDistrict').bfill().ffill()

# put Forestry information back in
FI['ForestryDistrict'] = fd
FI = FI[order]

In [38]:
FI

,ForestryDistrict,FederalState,Period,PeriodLength,Year,Area,DistrictShareAustrForest,DistrictShareStateForest,TotalForestShare,TotalForestArea,...,SpruceArea-SD,SpruceArea2,SpruceStockShare,SpruceStock,DWStShare,DWStStock,DWSt-SD,DeadSpruceStShare,DeadSpruceArea,DeadSpruceDensity
0,Eisenstadt+Rust,Burgenland,1992-1996,5,1996,516.03,0.004017,0.122137,0.310059,16000.000000,...,500.0,0.000000,0.213627,400000.0,0.028,6.800000,1.3,0.00280,3.706127,0.01652
1,Eisenstadt+Rust,Burgenland,1992-1996,5,1997,516.03,0.004033,0.122145,0.310867,16041.666667,...,500.0,146.520147,0.213627,400000.0,0.028,6.800000,1.3,0.00280,3.706127,0.01652
2,Eisenstadt+Rust,Burgenland,1992-1996,5,1998,516.03,0.004050,0.122152,0.311674,16083.333333,...,500.0,293.040293,0.213627,400000.0,0.028,6.800000,1.3,0.00280,3.706127,0.01652
3,Eisenstadt+Rust,Burgenland,1992-1996,5,1999,516.03,0.004066,0.122159,0.312482,16125.000000,...,500.0,439.560440,0.213627,400000.0,0.028,6.800000,1.3,0.00280,3.706127,0.01652
4,Eisenstadt+Rust,Burgenland,1992-1996,5,2000,516.03,0.004082,0.122166,0.313289,16166.666667,...,500.0,586.080586,0.213627,400000.0,0.028,6.800000,1.3,0.00280,3.706127,0.01652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2153,Vorarlberg,Vorarlberg_total,2007-2009,3,2017,2601.67,0.024404,1.000000,0.373612,97201.638667,...,4000.0,9285.115207,0.725265,15499000.0,0.025,10.633333,2.8,0.01595,462.550000,0.16588
2154,Vorarlberg,Vorarlberg_total,2007-2009,3,2018,2601.67,0.024411,1.000000,0.373709,97226.843500,...,4000.0,6963.836406,0.725265,15499000.0,0.025,10.575000,2.8,0.01595,462.550000,0.16588
2155,Vorarlberg,Vorarlberg_total,2007-2009,3,2019,2601.67,0.024417,1.000000,0.373806,97252.048333,...,4000.0,4642.557604,0.725265,15499000.0,0.025,10.516667,2.8,0.01595,462.550000,0.16588
2156,Vorarlberg,Vorarlberg_total,2007-2009,3,2020,2601.67,0.024423,1.000000,0.373903,97277.253167,...,4000.0,2321.278802,0.725265,15499000.0,0.025,10.458333,2.8,0.01595,462.550000,0.16588


# Export

In [39]:
FI.to_csv(f'{output_dir}/Data_BWF_ForestInventory_Postprocessed.csv')
print('Exported.')

Exported.
